In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
from bs4 import BeautifulSoup
import numpy as np
import requests
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
with open('vetting_playlist.json', 'r') as file:
    data1 = json.load(file)

In [3]:
data1.keys()

dict_keys(['collaborative', 'description', 'external_urls', 'followers', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])

In [4]:
file_paths = [
    'mpd.slice.15000-15999.json',
    'mpd.slice.16000-16999.json',
    'mpd.slice.17000-17999.json',
    'mpd.slice.18000-18999.json',
    'mpd.slice.19000-19999.json',
    'mpd.slice.20000-20999.json',
    'mpd.slice.21000-21999.json',
    'mpd.slice.22000-22999.json',
    'mpd.slice.23000-23999.json',
    'mpd.slice.24000-24999.json',
    'mpd.slice.25000-25999.json',
    'mpd.slice.26000-26999.json',
    'mpd.slice.27000-27999.json',
    'mpd.slice.28000-28999.json',
    'mpd.slice.29000-29999.json',
    'mpd.slice.30000-30999.json',
]

all_data = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)
        for playlist in data['playlists']:
            playlist_name = playlist['name']
            for track in playlist['tracks']:
                all_data.append({
                    "playlist_name": playlist_name,
                    "position_in_playlist": track["pos"],
                    "track_name": track['track_name'],
                    "artist_name": track['artist_name'],
                    "duration_ms": track['duration_ms'],
                    "album_name": track['album_name']
                })
df = pd.DataFrame(all_data)

In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077598 entries, 0 to 1077597
Data columns (total 6 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   playlist_name         1077598 non-null  object
 1   position_in_playlist  1077598 non-null  int64 
 2   track_name            1077598 non-null  object
 3   artist_name           1077598 non-null  object
 4   duration_ms           1077598 non-null  int64 
 5   album_name            1077598 non-null  object
dtypes: int64(2), object(4)
memory usage: 49.3+ MB


In [6]:
df.head()

,playlist_name,position_in_playlist,track_name,artist_name,duration_ms,album_name
0,Wedding Music,0,Always Forever,Phil Wickham,281880,Phil Wickham
1,Wedding Music,1,Divine Romance,Phil Wickham,298026,Phil Wickham
2,Wedding Music,2,Messiah / You're Beautiful,Phil Wickham,293720,Cannons
3,Wedding Music,3,Love Is Not A Fight,Warren Barfield,222986,Worth Fighting For
4,Wedding Music,4,When I Say I Do,Matthew West,246000,Hold You Up EP


In [7]:
track_name = df.drop_duplicates(subset=['track_name'])

In [8]:
track_name.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177749 entries, 0 to 1077595
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   playlist_name         177749 non-null  object
 1   position_in_playlist  177749 non-null  int64 
 2   track_name            177749 non-null  object
 3   artist_name           177749 non-null  object
 4   duration_ms           177749 non-null  int64 
 5   album_name            177749 non-null  object
dtypes: int64(2), object(4)
memory usage: 9.5+ MB


In [9]:
track_name.head(150)

,playlist_name,position_in_playlist,track_name,artist_name,duration_ms,album_name
0,Wedding Music,0,Always Forever,Phil Wickham,281880,Phil Wickham
1,Wedding Music,1,Divine Romance,Phil Wickham,298026,Phil Wickham
2,Wedding Music,2,Messiah / You're Beautiful,Phil Wickham,293720,Cannons
3,Wedding Music,3,Love Is Not A Fight,Warren Barfield,222986,Worth Fighting For
4,Wedding Music,4,When I Say I Do,Matthew West,246000,Hold You Up EP
...,...,...,...,...,...,...
147,PB,39,Chill Bill,Rob $tone,177184,Chill Bill
148,PB,40,Light It Up (feat. Nyla & Fuse ODG) - Remix,Major Lazer,166138,Peace Is The Mission: Extended
149,PB,41,Party Like We're Animals - Radio Edit,Klaas,191453,Party Like We're Animals
150,PB,42,This Is Dirty,DVBBS,295786,We Were Young


In [10]:
df_artists = df.drop_duplicates(subset=['artist_name'])

In [11]:
df_artists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45454 entries, 0 to 1077561
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   playlist_name         45454 non-null  object
 1   position_in_playlist  45454 non-null  int64 
 2   track_name            45454 non-null  object
 3   artist_name           45454 non-null  object
 4   duration_ms           45454 non-null  int64 
 5   album_name            45454 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.4+ MB


In [12]:
playlists = df.drop_duplicates(subset=['playlist_name'])

In [13]:
playlists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8650 entries, 0 to 1077551
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   playlist_name         8650 non-null   object
 1   position_in_playlist  8650 non-null   int64 
 2   track_name            8650 non-null   object
 3   artist_name           8650 non-null   object
 4   duration_ms           8650 non-null   int64 
 5   album_name            8650 non-null   object
dtypes: int64(2), object(4)
memory usage: 473.0+ KB
